# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [1]:
import sqlite3
import response_format
from dotenv import load_dotenv
from litellm import completion

load_dotenv()

connection = sqlite3.connect('../data/survey.db')
cursor = connection.cursor()

In [4]:
# strategie laden
cursor.execute("SELECT id, system_path, message_path FROM strategies WHERE name = 'oneshot_none'")
id, system_path, message_path = cursor.fetchone()

# prompts lesen
with open(f"../{system_path}", "r") as f:
    system_prompt = f.read()

with open(f"../{message_path}", "r") as f:
    message_prompt = f.read()

In [5]:
# fragen laden
cursor.execute("SELECT id, text FROM questions")
questions = cursor.fetchall()

# prompt anreichern
questions_text = "\n".join([f"{id}. {text}" for id, text in questions])
content = message_prompt.replace("{questions}", questions_text)

In [ ]:
# models laden
cursor.execute("SELECT id, model_id FROM models WHERE name = 'Mistral Small'")
id, model_id = cursor.fetchone()

In [8]:
reponse = completion(
    model = model_id,
    messages = [
        {"role": "user", "content": content}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "survey_response",
            "schema": response_format.CompleteResponseFormat.model_json_schema()
        }
    }
)

print(reponse)

ModelResponse(id='31120acccbe544489e5fd726a4b948de', created=1767123119, model='mistral-small-2506', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='{\n  "answers": [\n    {"question": 1, "answer": 4},\n    {"question": 2, "answer": 3},\n    {"question": 3, "answer": 3},\n    {"question": 4, "answer": 2},\n    {"question": 5, "answer": 3},\n    {"question": 6, "answer": 3},\n    {"question": 7, "answer": 2},\n    {"question": 8, "answer": 4},\n    {"question": 9, "answer": 2},\n    {"question": 10, "answer": 3},\n    {"question": 11, "answer": 4},\n    {"question": 12, "answer": 4},\n    {"question": 13, "answer": 4},\n    {"question": 14, "answer": 3},\n    {"question": 15, "answer": 3},\n    {"question": 16, "answer": 2},\n    {"question": 17, "answer": 4},\n    {"question": 18, "answer": 3},\n    {"question": 19, "answer": 3},\n    {"question": 20, "answer": 2},\n    {"question": 21, "answer": 4},\n    {"que

In [ ]:
# response in die datenbank schreiben

# Nach LLM-Call
parsed_response = rf.CompleteResponseFormat.model_validate_json(response.choices[0].message.content)

# In DB schreiben
for item in parsed_response.answers:
    cursor.execute(
        "INSERT INTO responses (model_id, strategy_id, question_id, answer) VALUES (?,?,?,?)",
        (model_id, strategy_id, item.question, item.answer)
    )

In [ ]:
# simple auswertung